In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [19]:
Corrected_Numbers = pd.read_csv("DOU CY2020 Perinatal PtDays.csv")
Q1 = Corrected_Numbers[Corrected_Numbers.Month.isin([1,2,3])]
Q2 = Corrected_Numbers[Corrected_Numbers.Month.isin([4,5,6])]
Q3 = Corrected_Numbers[Corrected_Numbers.Month.isin([7,8,9])]
Q4 = Corrected_Numbers[Corrected_Numbers.Month.isin([10,11,12])]

In [29]:
Perinatal_Volume = Q4

In [30]:
Perinatal_Volume

,Hospital,UnitName,Month,PatientDaysMethod,WomenPatientDaysCensus,NewbornPatientDaysCensus
0,EDH,EDH Mother/Baby,12,Method 5 - Patient Days from Multiple Census R...,614,437
2,EDH,EDH Mother/Baby,10,Method 5 - Patient Days from Multiple Census R...,612,503
11,EDH,EDH Mother/Baby,11,Method 5 - Patient Days from Multiple Census R...,534,461


In [8]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Perinatal_Pt_Days_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the Ambulatory Patient Volumes
 
    # The first step takes only the data for this selected hospital
    Hosp_data = Perinatal_Volume[Perinatal_Volume.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct year and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]


            for column in new_data.columns[3:]:

                new_value = new_data[column].iloc[idx2]
                month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Perinatal_Pt_Days.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

In [31]:
NDNQI_Perinatal_Pt_Days_XML('EDH','Perinatal PtDays Template.xml','NMW346',2020,4)